In [262]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.applications import VGG19
from tensorflow.keras import Model
import cv2 as cv

In [263]:
vgg19 = VGG19(include_top=False, weights='imagenet')
for layer in vgg19.layers:
  print(layer.name, layer.output_shape)

input_17 [(None, None, None, 3)]
block1_conv1 (None, None, None, 64)
block1_conv2 (None, None, None, 64)
block1_pool (None, None, None, 64)
block2_conv1 (None, None, None, 128)
block2_conv2 (None, None, None, 128)
block2_pool (None, None, None, 128)
block3_conv1 (None, None, None, 256)
block3_conv2 (None, None, None, 256)
block3_conv3 (None, None, None, 256)
block3_conv4 (None, None, None, 256)
block3_pool (None, None, None, 256)
block4_conv1 (None, None, None, 512)
block4_conv2 (None, None, None, 512)
block4_conv3 (None, None, None, 512)
block4_conv4 (None, None, None, 512)
block4_pool (None, None, None, 512)
block5_conv1 (None, None, None, 512)
block5_conv2 (None, None, None, 512)
block5_conv3 (None, None, None, 512)
block5_conv4 (None, None, None, 512)
block5_pool (None, None, None, 512)


In [264]:
content_layers = ['block4_conv2']
style_layers = ['block'+str(i)+'_conv1' for i in range(1,6)]

In [265]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  gram_matrix = tf.expand_dims(result, axis=0)
  input_shape = tf.shape(input_tensor)
  ij = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return gram_matrix/ij 

In [266]:
def load_vgg19():
  global vgg19,content_layers,style_layers
  vgg19.trainable = False
  content_output = vgg19.get_layer(style_layers[0]).output
  style_output = [vgg19.get_layer(layer).output for layer in style_layers]
  gram_style_output = [gram_matrix(output) for output in style_output]
  model = Model([vgg19.input],[content_output, gram_style_output])
  return model

In [267]:
opt = tf.optimizers.Adam(learning_rate=0.01, beta_1 = 0.99, epsilon = 1e-1) 

In [268]:
def loss_object(style_outputs, content_outputs, style_target, content_target):
  sweight = 1e-2
  cweight = 1e-1
  content_loss = tf.reduce_mean((content_outputs - content_target)**2)/2
  style_loss = tf.add_n([tf.reduce_mean((style_outputs[i]-style_target[i])**2) for i in range(len(style_outputs))])/2
  total_loss = sweight*style_loss + cweight*content_loss
  return total_loss

In [269]:
content_image = cv.imread('/content/image.jpeg')
style_image = cv.imread('/content/style.jpg')
content_image = cv.resize(content_image, (1024,1024))
style_image = cv.resize(style_image, (1024,1024))
content_image = np.array(content_image, np.float32)/255
style_image = np.array(style_image, np.float32)/255

In [270]:
vgg = load_vgg19()
content_target = vgg(np.array([content_image*255]))[0]
style_target = vgg(np.array([style_image*255]))[1]

In [271]:
def train_step(image, epoch):
  with tf.GradientTape() as tape:
    output = vgg(image*255)
    loss = loss_object(output[1], output[0], style_target, content_target)
  gradient = tape.gradient(loss, image)
  opt.apply_gradients([(gradient, image)])
  image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0))
  tf.print(f"Loss = {loss}")

In [272]:
EPOCHS = 300
image = tf.image.convert_image_dtype(content_image, tf.float32)
image = tf.Variable([image])
for i in range(EPOCHS):
  train_step(image, i)

Loss = 859114944.0
Loss = 641099200.0
Loss = 493174880.0
Loss = 391342976.0
Loss = 320678592.0
Loss = 272140352.0
Loss = 238797680.0
Loss = 216676288.0
Loss = 200725248.0
Loss = 188602032.0
Loss = 179855344.0
Loss = 173242048.0
Loss = 167671280.0
Loss = 162741008.0
Loss = 158213904.0
Loss = 153940224.0
Loss = 149828768.0
Loss = 145831600.0
Loss = 141828016.0
Loss = 137657920.0
Loss = 133261992.0
Loss = 128670248.0
Loss = 123984200.0
Loss = 119299344.0
Loss = 114726256.0
Loss = 110359632.0
Loss = 106249872.0
Loss = 102410120.0
Loss = 98847840.0
Loss = 95538136.0
Loss = 92450520.0
Loss = 89558144.0
Loss = 86846704.0
Loss = 84273176.0
Loss = 81838448.0
Loss = 79535128.0
Loss = 77372728.0
Loss = 75355408.0
Loss = 73481344.0
Loss = 71749136.0
Loss = 70139032.0
Loss = 68624800.0
Loss = 67191536.0
Loss = 65825388.0
Loss = 64511576.0
Loss = 63248252.0
Loss = 62034836.0
Loss = 60874276.0
Loss = 59756168.0
Loss = 58681692.0
Loss = 57648764.0
Loss = 56658828.0
Loss = 55706900.0
Loss = 54782864.0


In [273]:
final_img = np.array(image[0])

In [ ]:
plt.imshow(cv.cvtColor(final_img, cv.COLOR_BGR2RGB))

In [ ]:
final_img = final_img * 255
cv.imwrite('/content/final_image.png', final_img)